In [1]:
import bisect

In [2]:
class Index(object):
    
    def __init__(self, t, k):
        
        self.k = k
        self.index = []
        
        for i in range(len(t)-k+1):
            self.index.append((t[i:i+k], i))
        self.index.sort()
        
    def query(self,p):
        
        kmer = p[:self.k]
        i = bisect.bisect_left(self.index, (kmer, -1))
        hits = []
        while i < len(self.index):
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][0])
            i += 1
        return hits
                

In [3]:
def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets

In [8]:
from bm_preproc import *

In [9]:
BoyerMoore

bm_preproc.BoyerMoore

In [13]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [25]:
def aproximate_match(p,t,n):
    
    segment_lenght = round(len(p) / n + 1)
    all_matches = set()
    for i in range(n+1):
        start = i*segment_lenght
        end = min((i+1)*segment_lenght, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        print(p_bm.amap)
        matches = boyer_moore(p[start:end], p_bm, t)
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
                
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n:
                all_matches.add(m-start)
        return list(all_matches)

In [26]:
p = "AACTTG"
t = "CACTTAATTTG"
print(aproximate_match(p,t,2))

{'A': 0, 'C': 1, 'G': 2, 'T': 3}
[]
